<a href="https://colab.research.google.com/github/tztechno/cc_archive/blob/main/Gaussian_Splat_w_DINO%2BALIKED_06_xx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fountain: Gaussian Splat w/ DINO+ALIKED**

In [1]:

#サイズの異なる画像を扱う
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
import subprocess
import shutil
from pathlib import Path
import cv2
from PIL import Image

IMAGE_DIR = "/content/drive/MyDrive/your_folder/fountain2"
WORK_DIR = '/content/gaussian_splatting'
OUTPUT_DIR = '/content/output'
COLMAP_DIR = '/content/colmap_data'

ORIGINAL = IMAGE_DIR
RESIZED='/content/resized'

In [3]:
import os
import sys
import subprocess

def run_cmd(cmd, check=True, capture=False):
    """Run command with better error handling"""
    print(f"Running: {' '.join(cmd)}")
    result = subprocess.run(
        cmd,
        capture_output=capture,
        text=True,
        check=False
    )
    if check and result.returncode != 0:
        print(f"❌ Command failed with code {result.returncode}")
        if capture:
            print(f"STDOUT: {result.stdout}")
            print(f"STDERR: {result.stderr}")
    return result

def setup_environment():
    """
    Colab environment setup for Gaussian Splatting + LightGlue + pycolmap
    Python 3.12 compatible version (v8)
    """

    print("🚀 Setting up COLAB environment (v8 - Python 3.12 compatible)")

    WORK_DIR = "/content/gaussian-splatting"

    # =====================================================================
    # STEP 0: NumPy FIX (Python 3.12 compatible)
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 0: Fix NumPy (Python 3.12 compatible)")
    print("="*70)

    # Python 3.12 requires numpy >= 1.26
    run_cmd([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"])
    run_cmd([sys.executable, "-m", "pip", "install", "numpy==1.26.4"])

    # sanity check
    run_cmd([sys.executable, "-c", "import numpy; print('NumPy:', numpy.__version__)"])

    # =====================================================================
    # STEP 1: System packages (Colab)
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 1: System packages")
    print("="*70)

    run_cmd(["apt-get", "update", "-qq"])
    run_cmd([
        "apt-get", "install", "-y", "-qq",
        "colmap",
        "build-essential",
        "cmake",
        "git",
        "libopenblas-dev",
        "xvfb"
    ])

    # virtual display (COLMAP / OpenCV safety)
    os.environ["QT_QPA_PLATFORM"] = "offscreen"
    os.environ["DISPLAY"] = ":99"
    subprocess.Popen(
        ["Xvfb", ":99", "-screen", "0", "1024x768x24"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

    # =====================================================================
    # STEP 2: Clone Gaussian Splatting
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 2: Clone Gaussian Splatting")
    print("="*70)

    if not os.path.exists(WORK_DIR):
        run_cmd([
            "git", "clone", "--recursive",
            "https://github.com/graphdeco-inria/gaussian-splatting.git",
            WORK_DIR
        ])
    else:
        print("✓ Repository already exists")

    # =====================================================================
    # STEP 3: Python packages (FIXED ORDER & VERSIONS)
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 3: Python packages (VERBOSE MODE)")
    print("="*70)

    # ---- PyTorch (Colab CUDA対応) ----
    print("\n📦 Installing PyTorch...")
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "torch", "torchvision", "torchaudio"
    ])

    # ---- Core utils ----
    print("\n📦 Installing core utilities...")
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "opencv-python",
        "pillow",
        "imageio",
        "imageio-ffmpeg",
        "plyfile",
        "tqdm",
        "tensorboard"
    ])

    # ---- transformers (NumPy 1.26 compatible) ----
    print("\n📦 Installing transformers (NumPy 1.26 compatible)...")
    # Install transformers with proper dependencies
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "transformers==4.40.0"
    ])

    # ---- LightGlue stack (GITHUB INSTALL) ----
    print("\n📦 Installing LightGlue stack...")

    # Install kornia first
    run_cmd([sys.executable, "-m", "pip", "install", "kornia"])

    # Install h5py (sometimes needed)
    run_cmd([sys.executable, "-m", "pip", "install", "h5py"])

    # Install matplotlib (LightGlue dependency)
    run_cmd([sys.executable, "-m", "pip", "install", "matplotlib"])

    # Install LightGlue directly from GitHub (more reliable)
    print("  Installing LightGlue from GitHub...")
    run_cmd([sys.executable, "-m", "pip", "install",
            "git+https://github.com/cvg/LightGlue.git"])

    # Install pycolmap
    run_cmd([sys.executable, "-m", "pip", "install", "pycolmap"])

    # =====================================================================
    # STEP 4: Build GS submodules
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 4: Build Gaussian Splatting submodules")
    print("="*70)

    submodules = {
        "diff-gaussian-rasterization":
            "https://github.com/graphdeco-inria/diff-gaussian-rasterization.git",
        "simple-knn":
            "https://github.com/camenduru/simple-knn.git"
    }

    for name, repo in submodules.items():
        print(f"\n📦 Installing {name}...")
        path = os.path.join(WORK_DIR, "submodules", name)
        if not os.path.exists(path):
            run_cmd(["git", "clone", repo, path])
        run_cmd([sys.executable, "-m", "pip", "install", path])

    # =====================================================================
    # STEP 5: Detailed Verification
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 5: Detailed Verification")
    print("="*70)

    # NumPy (verify version first)
    print("\n🔍 Testing NumPy...")
    try:
        import numpy as np
        print(f"  ✓ NumPy: {np.__version__}")
    except Exception as e:
        print(f"  ❌ NumPy failed: {e}")

    # PyTorch
    print("\n🔍 Testing PyTorch...")
    try:
        import torch
        print(f"  ✓ PyTorch: {torch.__version__}")
        print(f"  ✓ CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"  ✓ CUDA version: {torch.version.cuda}")
    except Exception as e:
        print(f"  ❌ PyTorch failed: {e}")

    # transformers
    print("\n🔍 Testing transformers...")
    try:
        import transformers
        print(f"  ✓ transformers version: {transformers.__version__}")
        from transformers import AutoModel
        print(f"  ✓ AutoModel import: OK")
    except Exception as e:
        print(f"  ❌ transformers failed: {e}")
        print(f"  Attempting detailed diagnosis...")
        result = run_cmd([
            sys.executable, "-c",
            "import transformers; print(transformers.__version__)"
        ], capture=True)
        print(f"  Output: {result.stdout}")
        print(f"  Error: {result.stderr}")

    # LightGlue
    print("\n🔍 Testing LightGlue...")
    try:
        from lightglue import LightGlue, ALIKED
        print(f"  ✓ LightGlue: OK")
        print(f"  ✓ ALIKED: OK")
    except Exception as e:
        print(f"  ❌ LightGlue failed: {e}")
        print(f"  Attempting detailed diagnosis...")
        result = run_cmd([
            sys.executable, "-c",
            "from lightglue import LightGlue"
        ], capture=True)
        print(f"  Output: {result.stdout}")
        print(f"  Error: {result.stderr}")

    # pycolmap
    print("\n🔍 Testing pycolmap...")
    try:
        import pycolmap
        print(f"  ✓ pycolmap: OK")
    except Exception as e:
        print(f"  ❌ pycolmap failed: {e}")

    # kornia
    print("\n🔍 Testing kornia...")
    try:
        import kornia
        print(f"  ✓ kornia: {kornia.__version__}")
    except Exception as e:
        print(f"  ❌ kornia failed: {e}")

    print("\n" + "="*70)
    print("✅ SETUP COMPLETE")
    print("="*70)
    print(f"Working dir: {WORK_DIR}")

    return WORK_DIR


if __name__ == "__main__":
    setup_environment()

🚀 Setting up COLAB environment (v8 - Python 3.12 compatible)

STEP 0: Fix NumPy (Python 3.12 compatible)
Running: /usr/bin/python3 -m pip uninstall -y numpy
Running: /usr/bin/python3 -m pip install numpy==1.26.4
Running: /usr/bin/python3 -c import numpy; print('NumPy:', numpy.__version__)

STEP 1: System packages
Running: apt-get update -qq
Running: apt-get install -y -qq colmap build-essential cmake git libopenblas-dev xvfb

STEP 2: Clone Gaussian Splatting
Running: git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git /content/gaussian-splatting

STEP 3: Python packages (VERBOSE MODE)

📦 Installing PyTorch...
Running: /usr/bin/python3 -m pip install torch torchvision torchaudio

📦 Installing core utilities...
Running: /usr/bin/python3 -m pip install opencv-python pillow imageio imageio-ffmpeg plyfile tqdm tensorboard

📦 Installing transformers (NumPy 1.26 compatible)...
Running: /usr/bin/python3 -m pip install transformers==4.40.0

📦 Installing LightGlue sta

In [4]:
# setup successful 2026/01/07

In [5]:
def normalize_image_sizes(image_dir, output_dir=None, target_size=1024, mode='fit'):
    """
    Resizes all images in a directory while maintaining aspect ratio.

    Args:
        image_dir: Directory containing input images.
        output_dir: Directory to save the processed images. Defaults to image_dir.
        target_size: The desired maximum size for the longer side (or minimum size for the shorter side).
        mode: Resizing mode - 'fit' (fit within target), 'fill' (fill target), or 'pad' (fit with padding).
    """
    if output_dir is None:
        output_dir = image_dir

    os.makedirs(output_dir, exist_ok=True)

    print(f"Normalizing image sizes (mode: {mode}) while maintaining aspect ratio...")

    size_stats = {}
    converted_count = 0

    for img_file in sorted(os.listdir(image_dir)):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        input_path = os.path.join(image_dir, img_file)
        output_path = os.path.join(output_dir, img_file)

        try:
            img = Image.open(input_path)
            original_size = img.size  # (width, height)
            original_aspect = original_size[0] / original_size[1]

            # Record original size for statistics
            size_key = f"{original_size[0]}x{original_size[1]}"
            if size_key not in size_stats:
                size_stats[size_key] = 0
            size_stats[size_key] += 1

            # Resize while maintaining aspect ratio
            if mode == 'fit':
                # Fit within target (長辺をtarget_sizeに合わせてリサイズ)
                if original_size[0] > original_size[1]:  # 横長
                    new_width = target_size
                    new_height = int(target_size / original_aspect)
                else:  # 縦長 or 正方形
                    new_height = target_size
                    new_width = int(target_size * original_aspect)

            elif mode == 'fill':
                # Fill target (短辺をtarget_sizeに合わせてリサイズ)
                if original_size[0] > original_size[1]:  # 横長
                    new_height = target_size
                    new_width = int(target_size * original_aspect)
                else:  # 縦長 or 正方形
                    new_width = target_size
                    new_height = int(target_size / original_aspect)

            elif mode == 'pad':
                # Fit with padding (短辺をtarget_sizeに合わせて、余白を追加)
                if original_size[0] > original_size[1]:  # 横長
                    new_width = target_size
                    new_height = int(target_size / original_aspect)
                else:  # 縦長 or 正方形
                    new_height = target_size
                    new_width = int(target_size * original_aspect)

                # 余白を追加して正方形にする
                img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                img_square = Image.new('RGB', (target_size, target_size), (255, 255, 255))
                offset = ((target_size - new_width) // 2, (target_size - new_height) // 2)
                img_square.paste(img_resized, offset)
                img = img_square
                print(f"  ✓ {img_file}: {original_size} → {new_width}x{new_height} (padded to {target_size}x{target_size})")
                img.save(output_path, quality=95)
                converted_count += 1
                continue

            else:
                raise ValueError(f"Unknown mode: {mode}. Use 'fit', 'fill', or 'pad'.")

            # リサイズ実行
            img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            img_resized.save(output_path, quality=95)
            converted_count += 1

            print(f"  ✓ {img_file}: {original_size} → {new_width}x{new_height} (aspect ratio: {original_aspect:.2f})")

        except Exception as e:
            print(f"  ✗ Error processing {img_file}: {e}")

    print(f"\nConversion complete: {converted_count} images")
    print(f"Original size distribution: {size_stats}")
    return converted_count



converted_count=normalize_image_sizes(ORIGINAL, RESIZED, target_size=1024, mode='fit')
print(converted_count)


Normalizing image sizes (mode: fit) while maintaining aspect ratio...
  ✓ image_000.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_001.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_002.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_003.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_004.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_005.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_006.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_007.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_008.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_009.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_010.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_011.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_012.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_013.jpeg: (1440, 1920) → 768x1024 (aspect ratio: 0.75)
  ✓ image_014.jpeg: 

In [6]:
import os
import gc
import glob
import subprocess
from pathlib import Path
from tqdm import tqdm
import numpy as np
import h5py
import sqlite3
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from lightglue import ALIKED, LightGlue
from transformers import AutoImageProcessor, AutoModel
import pycolmap
from PIL import Image


class CONFIG:
    GLOBAL_TOPK = 200
    RATIO_THR = 1.2
    MATCH_THRESH = 10
    N_KEYPOINTS = 2048
    exhaustive_if_less = 20
    min_matches = 15
    max_num_keypoints = 8192
    image_size = 1024
    colmap_camera_model = 'SIMPLE_RADIAL'

In [10]:
def run_colmap_sequential(database_path, image_dir, output_dir):
    """Run COLMAP with adaptive focal length calibration and pair selection"""
    print("\n=== Stage 4: Running COLMAP Reconstruction ===")

    import sqlite3
    import numpy as np
    import cv2
    import h5py
    import re
    import os
    import subprocess
    from datetime import datetime, timezone

    # =====================================================================
    # STEP 1: Calibrate focal length (既存のまま)
    # =====================================================================
    print("\n🔧 Calibrating focal length from matches...")

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    cursor.execute("SELECT pair_id, rows FROM matches ORDER BY rows DESC LIMIT 1")
    pair_id, num_matches = cursor.fetchone()

    image_id2 = pair_id % 2147483648
    image_id1 = (pair_id - image_id2) // 2147483648

    cursor.execute("SELECT data FROM keypoints WHERE image_id = ?", (image_id1,))
    kpts1 = np.frombuffer(cursor.fetchone()[0], dtype=np.float32).reshape(-1, 2)

    cursor.execute("SELECT data FROM keypoints WHERE image_id = ?", (image_id2,))
    kpts2 = np.frombuffer(cursor.fetchone()[0], dtype=np.float32).reshape(-1, 2)

    cursor.execute("SELECT data FROM matches WHERE pair_id = ?", (pair_id,))
    matches = np.frombuffer(cursor.fetchone()[0], dtype=np.uint32).reshape(-1, 2)

    points1 = kpts1[matches[:, 0]]
    points2 = kpts2[matches[:, 1]]

    cursor.execute("SELECT width, height FROM cameras LIMIT 1")
    width, height = cursor.fetchone()
    cx, cy = width / 2, height / 2

    print(f"   Using pair with {num_matches} matches")
    print(f"   Image size: {width}x{height}")

    best_focal = None
    best_inliers = 0
    max_dim = max(width, height)

    for f in range(int(max_dim * 0.7), int(max_dim * 1.8), 50):
        pts1_norm = (points1 - np.array([cx, cy])) / f
        pts2_norm = (points2 - np.array([cx, cy])) / f

        E, mask = cv2.findEssentialMat(
            pts1_norm, pts2_norm,
            focal=1.0, pp=(0.0, 0.0),
            method=cv2.RANSAC,
            prob=0.9999,
            threshold=0.005
        )

        if mask is not None:
            inliers = int(np.sum(mask))
            if inliers > best_inliers:
                best_inliers = inliers
                best_focal = f

    if best_focal is None:
        best_focal = max_dim * 1.2
    else:
        print(f"   ✅ Calibrated focal length: {best_focal:.1f} pixels")
        print(f"      Inliers: {best_inliers}/{len(matches)} ({best_inliers/len(matches)*100:.1f}%)")

    params = np.array([best_focal, cx, cy, 0.0], dtype=np.float64)
    cursor.execute("UPDATE cameras SET params = ?", (params.tobytes(),))
    conn.commit()
    conn.close()

    print(f"   ✅ Database updated with focal={best_focal:.1f}")

    # =====================================================================
    # STEP 2: DATA-DRIVEN ADAPTIVE PAIR SELECTION
    # =====================================================================
    print("\n🔍 Analyzing matches for optimal initial pair...")

    matches_file = "/content/output/features/matches.h5"

    def get_image_number(filename):
        match = re.search(r'(\d+)', filename)
        return int(match.group(1)) if match else 0

    # Collect all pair statistics
    all_pairs = []
    with h5py.File(matches_file, 'r') as f:
        for key1 in f.keys():
            num1 = get_image_number(key1)
            for key2 in f[key1].keys():
                num2 = get_image_number(key2)
                matches_data = f[key1][key2][()]
                match_count = len(matches_data)
                distance = abs(num2 - num1)

                all_pairs.append({
                    'pair': (key1, key2),
                    'distance': distance,
                    'matches': match_count
                })

    if not all_pairs:
        raise RuntimeError("No pairs found in matches file")

    # Analyze dataset characteristics
    distances = np.array([p['distance'] for p in all_pairs])
    match_counts = np.array([p['matches'] for p in all_pairs])

    # Compute statistics
    match_median = np.median(match_counts)
    match_75th = np.percentile(match_counts, 75)
    match_90th = np.percentile(match_counts, 90)

    distance_median = np.median(distances)
    max_distance = np.max(distances)

    print(f"   Dataset statistics:")
    print(f"   - Total pairs: {len(all_pairs)}")
    print(f"   - Match count: median={match_median:.0f}, 75th={match_75th:.0f}, 90th={match_90th:.0f}")
    print(f"   - Distance: median={distance_median:.0f}, max={max_distance}")

    # Scoring function: balance between matches and distance
    # Prefer pairs with:
    # 1. High match count (relative to dataset)
    # 2. Medium-to-large distance (good baseline for triangulation)
    # 3. Avoid too-close pairs (poor geometry) and too-far pairs (may fail)

    def compute_score(pair_info):
        matches = pair_info['matches']
        distance = pair_info['distance']

        # Match score: normalize to 0-1 range
        match_score = min(matches / match_90th, 1.0)

        # Distance score: prefer middle-to-large distances
        # Penalize very close (<5% of max) and very far (>80% of max)
        min_distance = max(5, max_distance * 0.05)
        max_good_distance = max_distance * 0.8

        if distance < min_distance:
            distance_score = distance / min_distance * 0.5  # Penalty for too close
        elif distance > max_good_distance:
            distance_score = 0.7  # Slight penalty for very far
        else:
            # Sweet spot: linear increase from min to median
            distance_score = 0.5 + 0.5 * (distance - min_distance) / (distance_median - min_distance)
            distance_score = min(distance_score, 1.0)

        # Combined score: weighted average
        # Prioritize matches more than distance (70/30 split)
        return 0.7 * match_score + 0.3 * distance_score

    # Score all pairs
    for pair in all_pairs:
        pair['score'] = compute_score(pair)

    # Sort by score
    all_pairs.sort(key=lambda x: x['score'], reverse=True)

    # Filter: must have reasonable matches (at least 50th percentile)
    match_50th = np.percentile(match_counts, 50)
    candidates = [p for p in all_pairs if p['matches'] >= match_50th]

    if not candidates:
        print("   ⚠️  No pairs meet minimum criteria, using top scored pairs")
        candidates = all_pairs[:20]

    # Show top candidates
    print(f"\n📊 Top 10 candidate pairs:")
    for i, c in enumerate(candidates[:10], 1):
        print(f"   {i}. {c['pair'][0]} - {c['pair'][1]}: "
              f"{c['matches']} matches, distance={c['distance']}, score={c['score']:.3f}")

    # Select best pair
    best = candidates[0]
    best_pair = best['pair']

    print(f"\n✅ Selected pair: {best_pair[0]} - {best_pair[1]}")
    print(f"   Matches: {best['matches']}")
    print(f"   Distance: {best['distance']} images apart")
    print(f"   Score: {best['score']:.3f}")

    # Get image IDs from database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    cursor.execute("SELECT image_id FROM images WHERE name = ?", (best_pair[0],))
    result1 = cursor.fetchone()
    cursor.execute("SELECT image_id FROM images WHERE name = ?", (best_pair[1],))
    result2 = cursor.fetchone()

    conn.close()

    if not result1 or not result2:
        raise RuntimeError("Selected images not found in database")

    image_id1, image_id2 = result1[0], result2[0]
    print(f"   Image IDs: {image_id1}, {image_id2}")

    # =====================================================================
    # STEP 3: Run COLMAP with ULTRA-RELAXED parameters
    # =====================================================================
    os.makedirs(output_dir, exist_ok=True)

    env = os.environ.copy()
    env['QT_QPA_PLATFORM'] = 'offscreen'

    print(f"\n🚀 Starting mapper at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    print(f"💡 Using calibrated focal={best_focal:.1f}, ULTRA-RELAXED params")
    print()

    cmd_mapper = [
        'colmap', 'mapper',
        '--database_path', database_path,
        '--image_path', image_dir,
        '--output_path', output_dir,

        '--Mapper.init_image_id1', str(image_id1),
        '--Mapper.init_image_id2', str(image_id2),

        # ULTRA-RELAXED initialization
        '--Mapper.init_min_tri_angle', '1',
        '--Mapper.init_min_num_inliers', '30',
        '--Mapper.init_max_error', '12',

        '--Mapper.abs_pose_min_num_inliers', '15',
        '--Mapper.abs_pose_max_error', '15',
        '--Mapper.min_num_matches', '10',

        '--Mapper.ba_refine_focal_length', '1',
        '--Mapper.ba_refine_principal_point', '1',
        '--Mapper.ba_refine_extra_params', '1',
        '--Mapper.ba_global_max_num_iterations', '100',

        '--Mapper.filter_max_reproj_error', '12',
        '--Mapper.tri_min_angle', '1.0',

        '--Mapper.multiple_models', '0',
    ]

    process = subprocess.Popen(
        cmd_mapper,
        env=env,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )

    print("-" * 60)
    for line in iter(process.stdout.readline, ''):
        if line:
            print(line.rstrip(), flush=True)

    process.stdout.close()
    return_code = process.wait(timeout=3600)
    print("-" * 60)

    if return_code == 0:
        print(f"\n✅ COLMAP reconstruction saved to: {output_dir}")
        print(f"🕐 Completed at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")

        try:
            import pycolmap
            from pathlib import Path

            sparse_dirs = list(Path(output_dir).glob("*/"))
            if sparse_dirs:
                reconstruction = pycolmap.Reconstruction(str(sparse_dirs[0]))

                num_images = len(reconstruction.images)
                num_points = len(reconstruction.points3D)

                print("\n📊 Reconstruction Statistics:")
                print(f"   Registered images: {num_images}/{len(all_pairs)}")
                print(f"   3D points: {num_points:,}")

                if num_points > 0:
                    mean_error = reconstruction.compute_mean_reprojection_error()
                    print(f"   Mean reprojection error: {mean_error:.2f} pixels")

                    mean_track_length = sum(
                        len(p.track.elements) for p in reconstruction.points3D.values()
                    ) / num_points
                    print(f"   Mean track length: {mean_track_length:.1f}")

                for cam_id, camera in reconstruction.cameras.items():
                    print(f"\n📷 Camera {cam_id}:")
                    print(f"   Focal: {camera.focal_length:.1f} (initial: {best_focal:.1f})")
        except Exception as e:
            print(f"   ⚠️  Could not read reconstruction stats: {e}")
    else:
        print(f"\n❌ COLMAP mapper failed with return code {return_code}")
        raise subprocess.CalledProcessError(return_code, cmd_mapper)

In [11]:
def load_torch_image(fname, device=torch.device('cuda')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


def extract_dino_embeddings(fnames, device=torch.device('cuda')):
    print("\n=== Stage 1: Extracting DINO Global Features ===")

    processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
    model = AutoModel.from_pretrained('facebook/dinov2-base')
    model = model.eval().to(device)

    global_descs = []
    for img_path in tqdm(fnames, desc="DINO extraction"):
        timg = load_torch_image(img_path, device)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_feat = F.normalize(
                outputs.last_hidden_state[:,1:].max(dim=1)[0],
                dim=1, p=2
            )
        global_descs.append(dino_feat.detach().cpu())

    global_descs = torch.cat(global_descs, dim=0)
    print(f"Extracted global features: {global_descs.shape}")

    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

    return global_descs


def build_topk_pairs(global_feats, device):
    print("\n=== Building Top-K Pairs from Global Features ===")

    g = global_feats.to(device)
    sim = g @ g.T
    sim.fill_diagonal_(-1)

    N = sim.size(0)
    k = min(CONFIG.GLOBAL_TOPK, N - 1)
    k = max(k, 1)

    topk_indices = torch.topk(sim, k, dim=1).indices.cpu()

    pairs = set()
    for i, neighbors in enumerate(topk_indices):
        for j in neighbors:
            j = j.item()
            if i < j:
                pairs.add((i, j))

    pairs = sorted(list(pairs))
    print(f"Initial pairs from global features: {len(pairs)}")
    return pairs


def extract_aliked_features(fnames, device=torch.device('cuda')):
    print("\n=== Stage 2: Extracting ALIKED Local Features ===")

    dtype = torch.float32
    extractor = ALIKED(
        model_name="aliked-n16",
        max_num_keypoints=CONFIG.max_num_keypoints,
        detection_threshold=0.01,
        resize=CONFIG.image_size
    ).eval().to(device, dtype)

    keypoints_dict = {}
    descriptors_dict = {}

    for img_path in tqdm(fnames, desc="ALIKED extraction"):
        key = os.path.basename(img_path)
        image = load_torch_image(img_path, device=device).to(dtype)

        with torch.inference_mode():
            feats = extractor.extract(image)
            kpts = feats['keypoints'].reshape(-1, 2).detach().cpu()
            descs = feats['descriptors'].reshape(-1, 128).detach().cpu()
            descs = F.normalize(descs, dim=1).half()

        keypoints_dict[key] = kpts.numpy()
        descriptors_dict[key] = descs

    print(f"Extracted features for {len(keypoints_dict)} images")

    del extractor
    torch.cuda.empty_cache()
    gc.collect()

    return keypoints_dict, descriptors_dict


def verify_pairs_with_local_features(pairs, fnames, descriptors_dict, device):
    print("\n=== Verifying Pairs with Local Features ===")

    verified_pairs = []

    for i, j in tqdm(pairs, desc="Local verification"):
        key1 = os.path.basename(fnames[i])
        key2 = os.path.basename(fnames[j])

        desc1 = descriptors_dict[key1].to(device)
        desc2 = descriptors_dict[key2].to(device)

        if desc1.size(0) == 0 or desc2.size(0) == 0:
            continue

        with torch.inference_mode():
            sim = desc1 @ desc2.T
            nn1 = torch.argmax(sim, dim=1)
            nn2 = torch.argmax(sim, dim=0)
            mutual = torch.arange(len(nn1), device=device) == nn2[nn1]
            n_matches = mutual.sum().item()

        if n_matches >= CONFIG.MATCH_THRESH:
            verified_pairs.append((i, j))

    print(f"Verified pairs: {len(verified_pairs)}")
    return verified_pairs

In [12]:
def match_with_lightglue(verified_pairs, fnames, keypoints_dict, descriptors_dict,
                         output_dir, device=torch.device('cuda')):
    """Perform detailed matching using LightGlue - Fully Corrected Version"""
    print("\n=== Stage 3: Matching with LightGlue ===")

    os.makedirs(output_dir, exist_ok=True)

    lg_matcher = KF.LightGlueMatcher(
        "aliked", {
            "width_confidence": -1,
            "depth_confidence": -1,
            "mp": True if 'cuda' in str(device) else False
        }
    ).eval().to(device).half()

    print("Loaded LightGlue model")

    # Save keypoints
    kpts_h5_path = os.path.join(output_dir, 'keypoints.h5')
    with h5py.File(kpts_h5_path, 'w') as f:
        for img_path in fnames:
            key = os.path.basename(img_path)
            f.create_dataset(key, data=keypoints_dict[key])

    # Save matches
    matches_h5_path = os.path.join(output_dir, 'matches.h5')
    matched_pairs = 0
    skipped_pairs = 0
    total_matches = 0

    with h5py.File(matches_h5_path, 'w') as f_match:
        for i, j in tqdm(verified_pairs, desc="LightGlue matching"):
            key1 = os.path.basename(fnames[i])
            key2 = os.path.basename(fnames[j])

            kp1 = torch.from_numpy(keypoints_dict[key1]).to(device).half()
            kp2 = torch.from_numpy(keypoints_dict[key2]).to(device).half()
            desc1 = descriptors_dict[key1].to(device)
            desc2 = descriptors_dict[key2].to(device)

            if len(kp1) == 0 or len(kp2) == 0:
                skipped_pairs += 1
                continue

            with torch.inference_mode():
                try:
                    dists, idxs = lg_matcher(
                        desc1, desc2,
                        KF.laf_from_center_scale_ori(kp1[None]),
                        KF.laf_from_center_scale_ori(kp2[None])
                    )

                    # Check if matches were found
                    if idxs.numel() == 0:
                        skipped_pairs += 1
                        continue

                    # ★★★ Fix: Removed [0] ★★★
                    matches = idxs.cpu().numpy()  # (num_matches, 2)

                    # Check match count
                    num_matches = matches.shape[0]

                    if num_matches >= CONFIG.min_matches:
                        grp = f_match.require_group(key1)
                        grp.create_dataset(key2, data=matches)
                        matched_pairs += 1
                        total_matches += num_matches
                    else:
                        skipped_pairs += 1

                except Exception as e:
                    print(f"\nError matching {key1}-{key2}: {e}")
                    skipped_pairs += 1
                    continue

    del lg_matcher
    torch.cuda.empty_cache()
    gc.collect()

    print(f"\nMatching complete:")
    print(f"  Matched pairs: {matched_pairs}")
    print(f"  Skipped pairs: {skipped_pairs}")
    print(f"  Total matches: {total_matches}")
    print(f"  Average matches per pair: {total_matches/matched_pairs:.1f}" if matched_pairs > 0 else "")
    print(f"  Success rate: {matched_pairs/len(verified_pairs)*100:.1f}%")

    print(f"\nSaved keypoints to: {kpts_h5_path}")
    print(f"Saved matches to: {matches_h5_path}")




In [13]:
print('run colmap sequential')

run colmap sequential


In [14]:
def run_colmap_sequential(database_path, image_dir, output_dir):
    """Run COLMAP with proper camera model and database-driven pair selection"""
    print("\n=== Stage 4: Running COLMAP Reconstruction ===")

    import sqlite3
    import numpy as np
    import cv2
    import re
    import os
    import subprocess
    from datetime import datetime, timezone

    # =====================================================================
    # STEP 1: Calibrate focal length
    # =====================================================================
    print("\n🔧 Calibrating focal length from matches...")

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    cursor.execute("SELECT pair_id, rows FROM matches ORDER BY rows DESC LIMIT 1")
    pair_id, num_matches = cursor.fetchone()

    image_id2 = pair_id % 2147483648
    image_id1 = (pair_id - image_id2) // 2147483648

    cursor.execute("SELECT data FROM keypoints WHERE image_id = ?", (image_id1,))
    kpts1 = np.frombuffer(cursor.fetchone()[0], dtype=np.float32).reshape(-1, 2)

    cursor.execute("SELECT data FROM keypoints WHERE image_id = ?", (image_id2,))
    kpts2 = np.frombuffer(cursor.fetchone()[0], dtype=np.float32).reshape(-1, 2)

    cursor.execute("SELECT data FROM matches WHERE pair_id = ?", (pair_id,))
    matches = np.frombuffer(cursor.fetchone()[0], dtype=np.uint32).reshape(-1, 2)

    points1 = kpts1[matches[:, 0]]
    points2 = kpts2[matches[:, 1]]

    cursor.execute("SELECT width, height FROM cameras LIMIT 1")
    width, height = cursor.fetchone()
    cx, cy = width / 2, height / 2

    print(f"   Using pair with {num_matches} matches")
    print(f"   Image size: {width}x{height}")

    best_focal = None
    best_inliers = 0
    max_dim = max(width, height)

    for f in range(int(max_dim * 0.7), int(max_dim * 1.8), 50):
        pts1_norm = (points1 - np.array([cx, cy])) / f
        pts2_norm = (points2 - np.array([cx, cy])) / f

        E, mask = cv2.findEssentialMat(
            pts1_norm, pts2_norm,
            focal=1.0, pp=(0.0, 0.0),
            method=cv2.RANSAC,
            prob=0.9999,
            threshold=0.005
        )

        if mask is not None:
            inliers = int(np.sum(mask))
            if inliers > best_inliers:
                best_inliers = inliers
                best_focal = f

    if best_focal is None:
        best_focal = max_dim * 1.2
    else:
        print(f"   ✅ Calibrated focal length: {best_focal:.1f} pixels")
        print(f"      Inliers: {best_inliers}/{len(matches)} ({best_inliers/len(matches)*100:.1f}%)")

    # =====================================================================
    # STEP 1.5: Force PINHOLE model (fix PANORAMIC issue)
    # =====================================================================
    print("\n🔧 Setting PINHOLE camera model...")

    # PINHOLE model (1) requires: fx, fy, cx, cy
    params = np.array([best_focal, best_focal, cx, cy], dtype=np.float64)

    cursor.execute("UPDATE cameras SET model = ?, params = ?, prior_focal_length = ?",
                   (1, params.tobytes(), 1))

    # Reset all geometric verifications to force recalculation
    cursor.execute("UPDATE two_view_geometries SET config = 0")

    conn.commit()
    print(f"   ✅ Camera model: PINHOLE (model=1)")
    print(f"   ✅ Parameters: fx={best_focal:.1f}, fy={best_focal:.1f}, cx={cx:.1f}, cy={cy:.1f}")
    print(f"   ✅ Reset geometric verifications")

    # =====================================================================
    # STEP 2: Select pairs FROM DATABASE (not H5!)
    # =====================================================================
    print("\n🎯 Selecting initialization pairs FROM DATABASE...")

    def get_image_number(filename):
        match = re.search(r'(\d+)', filename)
        return int(match.group(1)) if match else 0

    # Get image name mapping
    cursor.execute("SELECT image_id, name FROM images")
    id_to_name = {row[0]: row[1] for row in cursor.fetchall()}

    # Get ALL matches from database
    cursor.execute("SELECT pair_id, rows FROM matches")
    all_matches = cursor.fetchall()

    pairs = []
    for pair_id, match_count in all_matches:
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648

        name1 = id_to_name.get(image_id1)
        name2 = id_to_name.get(image_id2)

        if name1 and name2:
            num1 = get_image_number(name1)
            num2 = get_image_number(name2)
            distance = abs(num2 - num1)

            pairs.append({
                'image_id1': image_id1,
                'image_id2': image_id2,
                'name1': name1,
                'name2': name2,
                'distance': distance,
                'matches': match_count
            })

    conn.close()

    # Statistics
    match_counts = [p['matches'] for p in pairs]
    match_75th = np.percentile(match_counts, 75)

    print(f"\n   Database statistics:")
    print(f"   - Total pairs: {len(pairs)}")
    print(f"   - 75th percentile matches: {match_75th:.0f}")

    # Filter: distance 10-25 with high match count
    candidates = [
        p for p in pairs
        if 10 <= p['distance'] <= 25
        and p['matches'] >= match_75th
    ]

    # Fallback if no candidates
    if not candidates:
        print("   ⚠️  Relaxing constraints...")
        candidates = [
            p for p in pairs
            if 8 <= p['distance'] <= 30
            and p['matches'] >= match_75th * 0.8
        ]

    # Sort by match count
    candidates.sort(key=lambda x: x['matches'], reverse=True)

    print(f"\n   Found {len(candidates)} candidates (distance 10-25, high matches)")

    print("\n📊 Top 10 candidates from DATABASE:")
    for i, c in enumerate(candidates[:10], 1):
        print(f"   {i:2d}. {c['name1']:15s} - {c['name2']:15s}: "
              f"{c['matches']:5d} matches, distance={c['distance']:2d}")

    if not candidates:
        raise RuntimeError("No suitable initialization pairs found in database")

    # =====================================================================
    # STEP 3: Try top candidates
    # =====================================================================
    os.makedirs(output_dir, exist_ok=True)

    env = os.environ.copy()
    env['QT_QPA_PLATFORM'] = 'offscreen'

    max_attempts = min(5, len(candidates))

    for attempt in range(max_attempts):
        candidate = candidates[attempt]

        print(f"\n{'='*70}")
        print(f"🎯 Attempt {attempt + 1}/{max_attempts}")
        print(f"   Pair: {candidate['name1']} - {candidate['name2']}")
        print(f"   Matches: {candidate['matches']}, Distance: {candidate['distance']}")
        print(f"   Image IDs: {candidate['image_id1']} <-> {candidate['image_id2']}")
        print(f"{'='*70}")

        print(f"\n🚀 Starting mapper at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
        print(f"💡 PINHOLE model, focal={best_focal:.1f}, RELAXED initialization")

        cmd_mapper = [
            'colmap', 'mapper',
            '--database_path', database_path,
            '--image_path', image_dir,
            '--output_path', output_dir,

            '--Mapper.init_image_id1', str(candidate['image_id1']),
            '--Mapper.init_image_id2', str(candidate['image_id2']),

            # RELAXED but not extreme
            '--Mapper.init_min_tri_angle', '1.0',
            '--Mapper.init_min_num_inliers', '50',     # Reasonable threshold
            '--Mapper.init_max_error', '8',             # Standard threshold

            '--Mapper.abs_pose_min_num_inliers', '20',
            '--Mapper.abs_pose_max_error', '12',
            '--Mapper.min_num_matches', '15',

            '--Mapper.ba_refine_focal_length', '1',
            '--Mapper.ba_refine_principal_point', '1',
            '--Mapper.ba_refine_extra_params', '1',
            '--Mapper.ba_global_max_num_iterations', '100',

            '--Mapper.filter_max_reproj_error', '8',
            '--Mapper.tri_min_angle', '1.5',

            '--Mapper.multiple_models', '0',
        ]

        process = subprocess.Popen(
            cmd_mapper,
            env=env,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        print("-" * 70)
        output_lines = []
        for line in iter(process.stdout.readline, ''):
            if line:
                print(line.rstrip(), flush=True)
                output_lines.append(line)

        process.stdout.close()
        return_code = process.wait(timeout=3600)
        print("-" * 70)

        initialization_failed = any("Initialization failed" in line for line in output_lines)

        if return_code == 0:
            print(f"\n✅ SUCCESS! Reconstruction saved to: {output_dir}")
            print(f"🕐 Completed at {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
            print(f"🎉 Used pair {attempt + 1}: {candidate['name1']} - {candidate['name2']}")

            try:
                import pycolmap
                from pathlib import Path

                sparse_dirs = list(Path(output_dir).glob("*/"))
                if sparse_dirs:
                    reconstruction = pycolmap.Reconstruction(str(sparse_dirs[0]))

                    num_images = len(reconstruction.images)
                    num_points = len(reconstruction.points3D)

                    print("\n📊 Reconstruction Statistics:")
                    print(f"   Registered images: {num_images}")
                    print(f"   3D points: {num_points:,}")

                    if num_points > 0:
                        mean_error = reconstruction.compute_mean_reprojection_error()
                        print(f"   Mean reprojection error: {mean_error:.2f} pixels")

                        mean_track_length = sum(
                            len(p.track.elements) for p in reconstruction.points3D.values()
                        ) / num_points
                        print(f"   Mean track length: {mean_track_length:.1f}")

                    for cam_id, camera in reconstruction.cameras.items():
                        print(f"\n📷 Camera {cam_id}:")
                        print(f"   Model: {camera.model.name}")
                        print(f"   Focal: {camera.focal_length:.1f} (initial: {best_focal:.1f})")
            except Exception as e:
                print(f"   ⚠️  Could not read reconstruction stats: {e}")

            return  # Success!

        elif initialization_failed:
            print(f"\n⚠️  Pair {attempt + 1} initialization failed, trying next candidate...")
        else:
            print(f"\n❌ Pair {attempt + 1} failed with error code {return_code}")

    # All attempts failed
    print(f"\n❌ All {max_attempts} candidates failed")
    print("\nPossible next steps:")
    print("  1. Check if images have sufficient overlap")
    print("  2. Try manual pair selection with COLMAP GUI")
    print("  3. Verify focal length calibration")
    raise RuntimeError(f"COLMAP failed after {max_attempts} attempts")

In [15]:
def import_into_colmap(image_dir, feature_dir, database_path):
    """Import with EXACT camera dimensions - NO GROUPING"""
    print("\n=== Creating COLMAP Database ===")

    if os.path.exists(database_path):
        os.remove(database_path)

    import cv2

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Create tables (same as before)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cameras (
            camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            model INTEGER NOT NULL,
            width INTEGER NOT NULL,
            height INTEGER NOT NULL,
            params BLOB,
            prior_focal_length INTEGER NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS images (
            image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name TEXT NOT NULL UNIQUE,
            camera_id INTEGER NOT NULL,
            prior_qw REAL,
            prior_qx REAL,
            prior_qy REAL,
            prior_qz REAL,
            prior_tx REAL,
            prior_ty REAL,
            prior_tz REAL,
            FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS keypoints (
            image_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS matches (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS two_view_geometries (
            pair_id INTEGER PRIMARY KEY NOT NULL,
            rows INTEGER NOT NULL,
            cols INTEGER NOT NULL,
            data BLOB,
            config INTEGER NOT NULL,
            F BLOB,
            E BLOB,
            H BLOB,
            qvec BLOB,
            tvec BLOB
        )
    """)

    kpts_file = os.path.join(feature_dir, 'keypoints.h5')
    matches_file = os.path.join(feature_dir, 'matches.h5')

    # 🔥 NO GROUPING - Use EXACT dimensions
    size_to_camera = {}
    fname_to_id = {}
    image_id = 1

    with h5py.File(kpts_file, 'r') as f:
        print(f"Importing {len(f.keys())} images...")

        for filename in tqdm(f.keys(), desc="Adding images"):
            image_path = os.path.join(image_dir, filename)
            try:
                img = Image.open(image_path)
                width, height = img.size  # 実際のサイズ: 768x1024
                img.close()
            except:
                continue

            # 🔥 Use EXACT size as key
            size_key = (width, height)

            if size_key not in size_to_camera:
                # 焦点距離は推測値（後で校正）
                focal = max(width, height) * 1.2
                params = np.array([focal, width/2, height/2, 0.0], dtype=np.float64)
                cursor.execute(
                    "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
                    (None, 2, width, height, params.tobytes(), 1)
                )
                size_to_camera[size_key] = cursor.lastrowid
                print(f"  Created camera: {width}x{height}, focal={focal:.0f}")

            camera_id = size_to_camera[size_key]
            cursor.execute(
                "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                (image_id, filename, camera_id, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
            )
            fname_to_id[filename] = image_id

            kpts = f[filename][()].astype(np.float32)
            if len(kpts.shape) == 1:
                kpts = kpts.reshape(-1, 2)
            cursor.execute(
                "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
                (image_id, kpts.shape[0], 2, kpts.tobytes())
            )
            image_id += 1

    print(f"\n✅ Created {len(size_to_camera)} camera(s) with EXACT dimensions")

    # Geometric verification (same as before)
    verified_count = 0

    with h5py.File(kpts_file, 'r') as f_kpts:
        with h5py.File(matches_file, 'r') as f_matches:
            print(f"\n🔧 Processing matches with geometric verification...")

            for key1 in tqdm(f_matches.keys(), desc="Verifying"):
                if key1 not in fname_to_id:
                    continue

                for key2 in f_matches[key1].keys():
                    if key2 not in fname_to_id:
                        continue

                    id1, id2 = fname_to_id[key1], fname_to_id[key2]
                    if id1 >= id2:
                        continue

                    matches = f_matches[key1][key2][()].astype(np.uint32)
                    if matches.shape[0] < 15:
                        continue

                    kpts1 = f_kpts[key1][()].astype(np.float64)
                    kpts2 = f_kpts[key2][()].astype(np.float64)

                    if len(kpts1.shape) == 1:
                        kpts1 = kpts1.reshape(-1, 2)
                    if len(kpts2.shape) == 1:
                        kpts2 = kpts2.reshape(-1, 2)

                    pts1 = kpts1[matches[:, 0]]
                    pts2 = kpts2[matches[:, 1]]

                    try:
                        F, mask = cv2.findFundamentalMat(
                            pts1, pts2,
                            cv2.FM_RANSAC,
                            3.0, 0.999
                        )

                        if F is None or mask is None:
                            continue

                        inliers = matches[mask.ravel() == 1]

                        if len(inliers) < 15:
                            continue

                        pair_id = id1 * 2147483648 + id2

                        cursor.execute(
                            "INSERT INTO matches VALUES (?, ?, ?, ?)",
                            (pair_id, len(inliers), 2, inliers.astype(np.uint32).tobytes())
                        )

                        cursor.execute(
                            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                            (pair_id, len(inliers), 2, inliers.astype(np.uint32).tobytes(),
                             2, F.astype(np.float64).tobytes(),
                             None, None, None, None)
                        )

                        verified_count += 1

                    except:
                        continue

    conn.commit()
    conn.close()

    print(f"\n✓ Database created: {database_path}")
    print(f"  Cameras: {len(size_to_camera)}")
    print(f"  Images: {len(fname_to_id)}")
    print(f"  ✅ Geometrically verified pairs: {verified_count}")

    return fname_to_id

In [16]:
def main_pipeline(image_dir, output_base_dir):
    """Complete pipeline"""

    from datetime import datetime, timezone

    print(f"\n🚀 Pipeline started at {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Get images
    img_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
    fnames = []
    for ext in img_extensions:
        fnames.extend(glob.glob(os.path.join(image_dir, ext)))
    fnames = sorted(fnames)
    print(f"\n📸 Found {len(fnames)} images")

    if len(fnames) == 0:
        raise ValueError("No images found!")

    # Create directories
    feature_dir = os.path.join(output_base_dir, 'features')
    colmap_dir = os.path.join(output_base_dir, 'colmap')
    sparse_dir = os.path.join(colmap_dir, 'sparse')
    os.makedirs(feature_dir, exist_ok=True)
    os.makedirs(colmap_dir, exist_ok=True)

    # Stages 1-3: Feature extraction and matching
    print(f"\n⏰ Stage 1 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    global_feats = extract_dino_embeddings(fnames, device)

    print(f"\n⏰ Stage 2 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    initial_pairs = build_topk_pairs(global_feats, device)
    keypoints_dict, descriptors_dict = extract_aliked_features(fnames, device)

    print(f"\n⏰ Stage 3 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    verified_pairs = verify_pairs_with_local_features(
        initial_pairs, fnames, descriptors_dict, device
    )
    match_with_lightglue(
        verified_pairs, fnames, keypoints_dict, descriptors_dict,
        feature_dir, device
    )

    from datetime import datetime, timezone
    print()
    print(datetime.now(timezone.utc))

    print(f"\n⏰ Stage 4 started: {datetime.now(timezone.utc).strftime('%H:%M:%S UTC')}")
    # Stage 4: COLMAP Database + Reconstruction
    database_path = os.path.join(colmap_dir, 'database.db')
    #import_into_colmap(image_dir, feature_dir, database_path)
    import_into_colmap(image_dir, feature_dir, database_path)
    run_colmap_sequential(database_path, image_dir, sparse_dir)

    print("\n" + "="*60)
    print(f"✅ Pipeline Complete!")
    print(f"🕐 Finished at {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")
    print("="*60)


main_pipeline(RESIZED, OUTPUT_DIR)


🚀 Pipeline started at 2026-01-07 17:19:38 UTC

📸 Found 80 images

⏰ Stage 1 started: 17:19:38 UTC

=== Stage 1: Extracting DINO Global Features ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

...

Adding images: 100%|██████████| 80/80 [00:00<00:00, 1774.16it/s]


  Created camera: 768x1024, focal=1229

✅ Created 1 camera(s) with EXACT dimensions

🔧 Processing matches with geometric verification...


Verifying: 100%|██████████| 79/79 [00:33<00:00,  2.39it/s]



✓ Database created: /content/output/colmap/database.db
  Cameras: 1
  Images: 80
  ✅ Geometrically verified pairs: 2876

=== Stage 4: Running COLMAP Reconstruction ===

🔧 Calibrating focal length from matches...
   Using pair with 6967 matches
   Image size: 768x1024
   ✅ Calibrated focal length: 716.0 pixels
      Inliers: 6967/6967 (100.0%)

🔧 Setting PINHOLE camera model...
   ✅ Camera model: PINHOLE (model=1)
   ✅ Parameters: fx=716.0, fy=716.0, cx=384.0, cy=512.0
   ✅ Reset geometric verifications

🎯 Selecting initialization pairs FROM DATABASE...

   Database statistics:
   - Total pairs: 2876
   - 75th percentile matches: 1975

   Found 73 candidates (distance 10-25, high matches)

📊 Top 10 candidates from DATABASE:
    1. image_037.jpeg  - image_047.jpeg :  2548 matches, distance=10
    2. image_035.jpeg  - image_045.jpeg :  2506 matches, distance=10
    3. image_038.jpeg  - image_048.jpeg :  2504 matches, distance=10
    4. image_039.jpeg  - image_049.jpeg :  2488 matches, di

RuntimeError: COLMAP failed after 5 attempts

In [ ]:
print('--------------debug--------------')

In [17]:
import sqlite3
import numpy as np

def fix_colmap_database(database_path):
    """Fix COLMAP database to use calibrated camera model instead of PANORAMIC"""

    print("\n" + "="*70)
    print("🔧 FIXING COLMAP DATABASE")
    print("="*70)

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # =====================================================================
    # 1. Check current camera model
    # =====================================================================
    print("\n1️⃣ Current camera configuration:")
    cursor.execute("SELECT camera_id, model, width, height, params FROM cameras")
    camera = cursor.fetchone()

    camera_id, model, width, height, params_blob = camera
    params = np.frombuffer(params_blob, dtype=np.float64)

    model_names = {
        0: "SIMPLE_PINHOLE",
        1: "PINHOLE",
        2: "SIMPLE_RADIAL",
        3: "RADIAL",
        4: "OPENCV",
        5: "OPENCV_FISHEYE",
        6: "FULL_OPENCV"
    }

    print(f"   Model: {model} ({model_names.get(model, 'UNKNOWN')})")
    print(f"   Size: {width}x{height}")
    print(f"   Parameters: {params}")

    # =====================================================================
    # 2. Force PINHOLE model with correct calibration
    # =====================================================================
    print("\n2️⃣ Forcing PINHOLE model (model=1)...")

    # PINHOLE model requires: fx, fy, cx, cy
    focal = params[0]  # Use calibrated focal length
    cx = width / 2
    cy = height / 2

    new_params = np.array([focal, focal, cx, cy], dtype=np.float64)

    # Update to PINHOLE model (1)
    cursor.execute(
        "UPDATE cameras SET model = ?, params = ?, prior_focal_length = ? WHERE camera_id = ?",
        (1, new_params.tobytes(), 1, camera_id)
    )

    print(f"   ✅ Updated to PINHOLE model")
    print(f"   Parameters: fx={focal:.1f}, fy={focal:.1f}, cx={cx:.1f}, cy={cy:.1f}")

    # =====================================================================
    # 3. Re-verify geometric relationships with correct model
    # =====================================================================
    print("\n3️⃣ Clearing PANORAMIC geometric verifications...")

    cursor.execute("SELECT COUNT(*) FROM two_view_geometries WHERE config = 2")
    panoramic_count = cursor.fetchone()[0]

    print(f"   Found {panoramic_count} PANORAMIC pairs")
    print(f"   Note: These will be re-computed by COLMAP with correct model")

    # Option 1: Delete all geometric verifications (COLMAP will recompute)
    # cursor.execute("DELETE FROM two_view_geometries")
    # print(f"   ✅ Deleted all geometric verifications")

    # Option 2: Just change configuration (faster)
    # Configuration 0 = uncalibrated, will force proper estimation
    cursor.execute("UPDATE two_view_geometries SET config = 0")
    print(f"   ✅ Reset all pairs to UNCALIBRATED (config=0)")

    conn.commit()
    conn.close()

    print("\n✅ Database fixed!")
    print("="*70)


def select_pairs_from_database(database_path):
    """Select initialization pairs from DATABASE (not H5)"""

    print("\n" + "="*70)
    print("🎯 SELECTING PAIRS FROM DATABASE")
    print("="*70)

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Get image name mapping
    cursor.execute("SELECT image_id, name FROM images")
    id_to_name = {row[0]: row[1] for row in cursor.fetchall()}
    name_to_id = {row[1]: row[0] for row in cursor.fetchall()}

    # Get matches from database (what COLMAP actually sees)
    cursor.execute("SELECT pair_id, rows FROM matches ORDER BY rows DESC")
    all_matches = cursor.fetchall()

    # Parse pairs with distances
    import re
    def get_image_number(filename):
        match = re.search(r'(\d+)', filename)
        return int(match.group(1)) if match else 0

    pairs = []
    for pair_id, match_count in all_matches:
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648

        name1 = id_to_name.get(image_id1)
        name2 = id_to_name.get(image_id2)

        if name1 and name2:
            num1 = get_image_number(name1)
            num2 = get_image_number(name2)
            distance = abs(num2 - num1)

            pairs.append({
                'image_id1': image_id1,
                'image_id2': image_id2,
                'name1': name1,
                'name2': name2,
                'distance': distance,
                'matches': match_count
            })

    conn.close()

    # =====================================================================
    # Filter for good initialization pairs
    # =====================================================================
    print(f"\n   Total pairs in database: {len(pairs)}")

    # Strategy: Find pairs with:
    # 1. Good baseline (distance 10-25)
    # 2. High match count (top 25%)
    match_counts = [p['matches'] for p in pairs]
    match_75th = np.percentile(match_counts, 75)

    print(f"   75th percentile matches: {match_75th:.0f}")

    # Filter candidates
    candidates = [
        p for p in pairs
        if 10 <= p['distance'] <= 25  # Good baseline
        and p['matches'] >= match_75th  # High quality
    ]

    # Sort by match count
    candidates.sort(key=lambda x: x['matches'], reverse=True)

    print(f"\n   Candidates (distance 10-25, high matches): {len(candidates)}")

    if not candidates:
        print("   ⚠️  No candidates found, relaxing distance constraint...")
        candidates = [
            p for p in pairs
            if 8 <= p['distance'] <= 30
            and p['matches'] >= match_75th * 0.8
        ]
        candidates.sort(key=lambda x: x['matches'], reverse=True)

    print("\n📊 Top 10 candidates from DATABASE:")
    print("   " + "-"*70)
    for i, c in enumerate(candidates[:10], 1):
        print(f"   {i:2d}. {c['name1']:15s} - {c['name2']:15s}: "
              f"{c['matches']:5d} matches, distance={c['distance']:2d}")
        print(f"       Image IDs: {c['image_id1']} <-> {c['image_id2']}")

    print("="*70)

    return candidates[:5]  # Return top 5


# =====================================================================
# USAGE
# =====================================================================
database_path = "/content/output/colmap/database.db"

# Step 1: Fix the database
fix_colmap_database(database_path)

# Step 2: Get correct pairs
candidates = select_pairs_from_database(database_path)

print("\n🎯 Use these image IDs for initialization:")
for i, c in enumerate(candidates, 1):
    print(f"{i}. --Mapper.init_image_id1 {c['image_id1']} --Mapper.init_image_id2 {c['image_id2']}")


🔧 FIXING COLMAP DATABASE

1️⃣ Current camera configuration:
   Model: 1 (PINHOLE)
   Size: 768x1024
   Parameters: [716. 716. 384. 512.]

2️⃣ Forcing PINHOLE model (model=1)...
   ✅ Updated to PINHOLE model
   Parameters: fx=716.0, fy=716.0, cx=384.0, cy=512.0

3️⃣ Clearing PANORAMIC geometric verifications...
   Found 0 PANORAMIC pairs
   Note: These will be re-computed by COLMAP with correct model
   ✅ Reset all pairs to UNCALIBRATED (config=0)

✅ Database fixed!

🎯 SELECTING PAIRS FROM DATABASE

   Total pairs in database: 2876
   75th percentile matches: 1975

   Candidates (distance 10-25, high matches): 73

📊 Top 10 candidates from DATABASE:
   ----------------------------------------------------------------------
    1. image_037.jpeg  - image_047.jpeg :  2548 matches, distance=10
       Image IDs: 38 <-> 48
    2. image_035.jpeg  - image_045.jpeg :  2506 matches, distance=10
       Image IDs: 36 <-> 46
    3. image_038.jpeg  - image_048.jpeg :  2504 matches, distance=10
      

In [18]:
import sqlite3
import numpy as np

def inspect_colmap_database(database_path):
    """Inspect COLMAP database structure and sample data"""

    print("\n" + "="*70)
    print("🔍 COLMAP DATABASE INSPECTION")
    print("="*70)

    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # =====================================================================
    # 1. CAMERAS TABLE
    # =====================================================================
    print("\n1️⃣ CAMERAS TABLE")
    print("-"*70)

    cursor.execute("SELECT * FROM cameras")
    cameras = cursor.fetchall()

    print(f"Total cameras: {len(cameras)}")
    for cam in cameras:
        camera_id, model, width, height, params_blob, prior_focal_length = cam
        params = np.frombuffer(params_blob, dtype=np.float64)

        print(f"\nCamera ID: {camera_id}")
        print(f"  Model: {model} (0=SIMPLE_PINHOLE, 1=PINHOLE, 2=SIMPLE_RADIAL, 3=RADIAL)")
        print(f"  Image size: {width} x {height}")
        print(f"  Parameters (raw binary -> float64 array):")
        print(f"    {params}")
        print(f"  Interpretation:")
        if model == 0:  # SIMPLE_PINHOLE
            print(f"    - focal length (f): {params[0]:.2f}")
            print(f"    - principal point (cx, cy): ({params[1]:.2f}, {params[2]:.2f})")
        elif model == 1:  # PINHOLE
            print(f"    - focal lengths (fx, fy): ({params[0]:.2f}, {params[1]:.2f})")
            print(f"    - principal point (cx, cy): ({params[2]:.2f}, {params[3]:.2f})")
        print(f"  Prior focal length: {prior_focal_length}")

    # =====================================================================
    # 2. IMAGES TABLE
    # =====================================================================
    print("\n2️⃣ IMAGES TABLE (sample)")
    print("-"*70)

    cursor.execute("SELECT COUNT(*) FROM images")
    total_images = cursor.fetchone()[0]
    print(f"Total images: {total_images}")

    cursor.execute("SELECT * FROM images LIMIT 5")
    images = cursor.fetchall()

    print("\nFirst 5 images:")
    for img in images:
        image_id, name, camera_id, prior_qw, prior_qx, prior_qy, prior_qz, prior_tx, prior_ty, prior_tz = img
        print(f"\n  Image ID: {image_id}")
        print(f"    Name: {name}")
        print(f"    Camera ID: {camera_id}")
        print(f"    Prior pose (quaternion): [{prior_qw:.3f}, {prior_qx:.3f}, {prior_qy:.3f}, {prior_qz:.3f}]")
        print(f"    Prior translation: [{prior_tx:.3f}, {prior_ty:.3f}, {prior_tz:.3f}]")

    # =====================================================================
    # 3. KEYPOINTS TABLE
    # =====================================================================
    print("\n3️⃣ KEYPOINTS TABLE (sample)")
    print("-"*70)

    cursor.execute("SELECT COUNT(*) FROM keypoints")
    total_kpts = cursor.fetchone()[0]
    print(f"Total keypoint records: {total_kpts}")

    cursor.execute("SELECT image_id, data FROM keypoints LIMIT 3")
    keypoints = cursor.fetchall()

    print("\nFirst 3 keypoint records:")
    for image_id, kpts_blob in keypoints:
        kpts = np.frombuffer(kpts_blob, dtype=np.float32).reshape(-1, 2)
        print(f"\n  Image ID: {image_id}")
        print(f"    Total keypoints: {len(kpts)}")
        print(f"    Data format: float32 array, shape=({len(kpts)}, 2)")
        print(f"    First 5 keypoints (x, y):")
        for i, kpt in enumerate(kpts[:5]):
            print(f"      [{i}]: ({kpt[0]:.2f}, {kpt[1]:.2f})")
        print(f"    Statistics:")
        print(f"      X range: [{kpts[:, 0].min():.1f}, {kpts[:, 0].max():.1f}]")
        print(f"      Y range: [{kpts[:, 1].min():.1f}, {kpts[:, 1].max():.1f}]")

    # =====================================================================
    # 4. MATCHES TABLE
    # =====================================================================
    print("\n4️⃣ MATCHES TABLE (sample)")
    print("-"*70)

    cursor.execute("SELECT COUNT(*) FROM matches")
    total_matches = cursor.fetchone()[0]
    print(f"Total match records: {total_matches}")

    # Get top matches by count
    cursor.execute("SELECT pair_id, rows, data FROM matches ORDER BY rows DESC LIMIT 5")
    matches = cursor.fetchall()

    # Get image names
    cursor.execute("SELECT image_id, name FROM images")
    id_to_name = {row[0]: row[1] for row in cursor.fetchall()}

    print("\nTop 5 matches by count:")
    for pair_id, num_matches, matches_blob in matches:
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648

        name1 = id_to_name.get(image_id1, f"ID_{image_id1}")
        name2 = id_to_name.get(image_id2, f"ID_{image_id2}")

        matches_data = np.frombuffer(matches_blob, dtype=np.uint32).reshape(-1, 2)

        print(f"\n  Pair: {name1} <-> {name2}")
        print(f"    Image IDs: {image_id1} <-> {image_id2}")
        print(f"    Pair ID (encoded): {pair_id}")
        print(f"    Match count: {num_matches}")
        print(f"    Data format: uint32 array, shape=({len(matches_data)}, 2)")
        print(f"    First 5 matches (kpt_idx1, kpt_idx2):")
        for i, match in enumerate(matches_data[:5]):
            print(f"      [{i}]: keypoint {match[0]} <-> keypoint {match[1]}")

    # =====================================================================
    # 5. TWO_VIEW_GEOMETRIES TABLE (CRITICAL!)
    # =====================================================================
    print("\n5️⃣ TWO_VIEW_GEOMETRIES TABLE (geometric verification)")
    print("-"*70)

    cursor.execute("SELECT COUNT(*) FROM two_view_geometries")
    total_geom = cursor.fetchone()[0]
    print(f"Total geometry records: {total_geom}")

    cursor.execute("SELECT pair_id, rows, data, config, F, E, H FROM two_view_geometries ORDER BY rows DESC LIMIT 5")
    geometries = cursor.fetchall()

    print("\nTop 5 verified pairs by inlier count:")
    for pair_id, inlier_count, inliers_blob, config, F_blob, E_blob, H_blob in geometries:
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648

        name1 = id_to_name.get(image_id1, f"ID_{image_id1}")
        name2 = id_to_name.get(image_id2, f"ID_{image_id2}")

        print(f"\n  Pair: {name1} <-> {name2}")
        print(f"    Image IDs: {image_id1} <-> {image_id2}")
        print(f"    ✅ Inlier count: {inlier_count}")
        print(f"    Configuration: {config} (1=PLANAR, 2=PANORAMIC, 3=PLANAR_OR_PANORAMIC, 4-7=UNCALIBRATED)")

        # Decode inliers (uint32 array of match indices)
        if inliers_blob:
            inliers = np.frombuffer(inliers_blob, dtype=np.uint32)
            print(f"    Inlier indices (first 10): {inliers[:10]}")

        # Decode matrices
        if F_blob:
            F = np.frombuffer(F_blob, dtype=np.float64).reshape(3, 3)
            print(f"    Fundamental matrix F (3x3):")
            print(f"      {F[0]}")
            print(f"      {F[1]}")
            print(f"      {F[2]}")

        if E_blob:
            E = np.frombuffer(E_blob, dtype=np.float64).reshape(3, 3)
            print(f"    Essential matrix E available: Yes")

    # =====================================================================
    # 6. CRITICAL COMPARISON
    # =====================================================================
    print("\n6️⃣ CRITICAL COMPARISON: Matches vs Verified Inliers")
    print("-"*70)

    cursor.execute("""
        SELECT m.pair_id, m.rows as match_count, g.rows as inlier_count
        FROM matches m
        LEFT JOIN two_view_geometries g ON m.pair_id = g.pair_id
        ORDER BY m.rows DESC
        LIMIT 10
    """)

    comparison = cursor.fetchall()

    print("\nTop 10 pairs - Match count vs Verified inlier count:")
    print(f"{'Pair ID':<20} | {'Image 1':<15} | {'Image 2':<15} | {'Matches':<8} | {'Inliers':<8} | {'Loss %'}")
    print("-"*95)

    for pair_id, match_count, inlier_count in comparison:
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648

        name1 = id_to_name.get(image_id1, f"ID_{image_id1}")
        name2 = id_to_name.get(image_id2, f"ID_{image_id2}")

        if inlier_count:
            loss_pct = (match_count - inlier_count) / match_count * 100
            print(f"{pair_id:<20} | {name1:<15} | {name2:<15} | {match_count:<8} | {inlier_count:<8} | {loss_pct:>6.1f}%")
        else:
            print(f"{pair_id:<20} | {name1:<15} | {name2:<15} | {match_count:<8} | {'N/A':<8} | {'N/A'}")

    conn.close()

    print("\n" + "="*70)
    print("✅ Inspection complete!")
    print("="*70)


# Usage
database_path = "/content/output/colmap/database.db"
inspect_colmap_database(database_path)


🔍 COLMAP DATABASE INSPECTION

1️⃣ CAMERAS TABLE
----------------------------------------------------------------------
Total cameras: 1

Camera ID: 1
  Model: 1 (0=SIMPLE_PINHOLE, 1=PINHOLE, 2=SIMPLE_RADIAL, 3=RADIAL)
  Image size: 768 x 1024
  Parameters (raw binary -> float64 array):
    [716. 716. 384. 512.]
  Interpretation:
    - focal lengths (fx, fy): (716.00, 716.00)
    - principal point (cx, cy): (384.00, 512.00)
  Prior focal length: 1

2️⃣ IMAGES TABLE (sample)
----------------------------------------------------------------------
Total images: 80

First 5 images:

  Image ID: 1
    Name: image_000.jpeg
    Camera ID: 1
    Prior pose (quaternion): [1.000, 0.000, 0.000, 0.000]
    Prior translation: [0.000, 0.000, 0.000]

  Image ID: 2
    Name: image_001.jpeg
    Camera ID: 1
    Prior pose (quaternion): [1.000, 0.000, 0.000, 0.000]
    Prior translation: [0.000, 0.000, 0.000]

  Image ID: 3
    Name: image_002.jpeg
    Camera ID: 1
    Prior pose (quaternion): [1.000, 0.

In [ ]:
import sqlite3
import h5py
import re
import numpy as np

def debug_database_vs_h5(database_path, matches_h5_path):
    """Compare matches in database.db vs matches.h5"""

    print("\n" + "="*70)
    print("🔍 DEBUGGING: Database vs H5 Match Counts")
    print("="*70)

    def get_image_number(filename):
        match = re.search(r'(\d+)', filename)
        return int(match.group(1)) if match else 0

    # Read H5 matches
    print("\n1️⃣ Reading matches.h5...")
    h5_pairs = {}
    with h5py.File(matches_h5_path, 'r') as f:
        for key1 in f.keys():
            for key2 in f[key1].keys():
                matches = f[key1][key2][()]
                pair_key = tuple(sorted([key1, key2]))
                h5_pairs[pair_key] = len(matches)

    print(f"   Found {len(h5_pairs)} pairs in H5")
    print(f"   Total H5 matches: {sum(h5_pairs.values()):,}")

    # Read database matches
    print("\n2️⃣ Reading database.db...")
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Get image name mapping
    cursor.execute("SELECT image_id, name FROM images")
    id_to_name = {row[0]: row[1] for row in cursor.fetchall()}

    # Get matches from database
    cursor.execute("SELECT pair_id, rows FROM matches")
    db_matches = cursor.fetchall()

    db_pairs = {}
    for pair_id, num_matches in db_matches:
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648

        name1 = id_to_name.get(image_id1)
        name2 = id_to_name.get(image_id2)

        if name1 and name2:
            pair_key = tuple(sorted([name1, name2]))
            db_pairs[pair_key] = num_matches

    conn.close()

    print(f"   Found {len(db_pairs)} pairs in database")
    print(f"   Total DB matches: {sum(db_pairs.values()):,}")

    # Compare
    print("\n3️⃣ Comparing differences...")

    # Find pairs only in H5
    only_h5 = set(h5_pairs.keys()) - set(db_pairs.keys())
    # Find pairs only in DB
    only_db = set(db_pairs.keys()) - set(h5_pairs.keys())
    # Find common pairs with different counts
    common = set(h5_pairs.keys()) & set(db_pairs.keys())

    print(f"\n   Pairs only in H5: {len(only_h5)}")
    print(f"   Pairs only in DB: {len(only_db)}")
    print(f"   Common pairs: {len(common)}")

    # Analyze differences in common pairs
    differences = []
    for pair in common:
        h5_count = h5_pairs[pair]
        db_count = db_pairs[pair]
        if h5_count != db_count:
            diff = h5_count - db_count
            diff_pct = (diff / h5_count * 100) if h5_count > 0 else 0
            differences.append({
                'pair': pair,
                'h5': h5_count,
                'db': db_count,
                'diff': diff,
                'diff_pct': diff_pct,
                'distance': abs(get_image_number(pair[0]) - get_image_number(pair[1]))
            })

    if differences:
        print(f"\n   ⚠️  {len(differences)} pairs have different match counts!")

        # Sort by absolute difference
        differences.sort(key=lambda x: abs(x['diff']), reverse=True)

        print("\n   Top 10 largest discrepancies:")
        print("   " + "-"*65)
        print("   Pair                              | H5    | DB    | Diff  | %")
        print("   " + "-"*65)
        for i, d in enumerate(differences[:10], 1):
            print(f"   {d['pair'][0]:15s} - {d['pair'][1]:15s} | "
                  f"{d['h5']:5d} | {d['db']:5d} | {d['diff']:5d} | {d['diff_pct']:+5.1f}%")

        # Statistics
        diffs = [d['diff'] for d in differences]
        print(f"\n   Difference statistics:")
        print(f"   - Mean difference: {np.mean(diffs):.1f} matches")
        print(f"   - Median difference: {np.median(diffs):.1f} matches")
        print(f"   - Max loss: {min(diffs)} matches")
        print(f"   - Max gain: {max(diffs)} matches")

        # Analyze by distance
        print(f"\n4️⃣ Analyzing differences by distance...")
        distance_groups = {}
        for d in differences:
            dist = d['distance']
            if dist not in distance_groups:
                distance_groups[dist] = []
            distance_groups[dist].append(d['diff'])

        print("\n   Distance | Pairs | Avg Diff | Avg Loss %")
        print("   " + "-"*50)
        for dist in sorted(distance_groups.keys())[:20]:
            diffs = distance_groups[dist]
            avg_diff = np.mean(diffs)
            # Calculate average loss percentage
            relevant_diffs = [d for d in differences if d['distance'] == dist]
            avg_loss_pct = np.mean([d['diff_pct'] for d in relevant_diffs])
            print(f"   {dist:3d}      | {len(diffs):5d} | {avg_diff:+8.1f} | {avg_loss_pct:+8.1f}%")
    else:
        print("   ✅ All common pairs have identical match counts")

    # Find top pairs in DB that should be used for initialization
    print("\n5️⃣ Top pairs in DATABASE (what COLMAP actually sees):")
    print("   " + "-"*70)

    db_pairs_with_dist = []
    for pair, count in db_pairs.items():
        dist = abs(get_image_number(pair[0]) - get_image_number(pair[1]))
        db_pairs_with_dist.append({
            'pair': pair,
            'matches': count,
            'distance': dist
        })

    # Sort by match count
    db_pairs_with_dist.sort(key=lambda x: x['matches'], reverse=True)

    print("   Top 20 pairs by match count in DATABASE:")
    for i, p in enumerate(db_pairs_with_dist[:20], 1):
        print(f"   {i:2d}. {p['pair'][0]:15s} - {p['pair'][1]:15s}: "
              f"{p['matches']:5d} matches, distance={p['distance']:2d}")

    # Filter for good initialization pairs (distance 8-20, high matches)
    good_pairs = [p for p in db_pairs_with_dist
                  if 8 <= p['distance'] <= 20 and p['matches'] >= 1000]

    if good_pairs:
        print(f"\n   🎯 RECOMMENDED pairs (distance 8-20, 1000+ matches in DB):")
        for i, p in enumerate(good_pairs[:10], 1):
            print(f"   {i:2d}. {p['pair'][0]:15s} - {p['pair'][1]:15s}: "
                  f"{p['matches']:5d} matches, distance={p['distance']:2d}")

    print("\n" + "="*70)


# Usage
database_path = "/content/output/colmap/database.db"
matches_h5_path = "/content/output/features/matches.h5"

debug_database_vs_h5(database_path, matches_h5_path)

In [ ]:
import h5py
import re

matches_file = "/content/output/features/matches.h5"

def get_image_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else 0

# 距離別の統計
distance_stats = {}

with h5py.File(matches_file, 'r') as f:
    for key1 in f.keys():
        num1 = get_image_number(key1)
        for key2 in f[key1].keys():
            num2 = get_image_number(key2)
            matches = f[key1][key2][()]
            distance = abs(num2 - num1)

            if distance not in distance_stats:
                distance_stats[distance] = []
            distance_stats[distance].append(len(matches))

print("📊 距離別マッチ数統計:")
print("距離 | ペア数 | 平均マッチ | 最大マッチ | 500以上のペア数")
print("-" * 70)

for dist in sorted(distance_stats.keys()):
    if dist > 50:  # 最初の50距離だけ
        break

    matches_list = distance_stats[dist]
    avg = sum(matches_list) / len(matches_list)
    max_m = max(matches_list)
    count_500plus = sum(1 for m in matches_list if m >= 500)

    marker = ""
    if 20 <= dist <= 40 and count_500plus > 0:
        marker = " ← 理想的"
    elif 10 <= dist <= 19 and count_500plus > 0:
        marker = " ← 可能"

    print(f"{dist:3d}  | {len(matches_list):4d}   | {avg:7.1f}    | {max_m:6d}    | {count_500plus:6d}{marker}")

# 最も離れたペアで500以上のマッチがあるものを探す
print("\n📍 距離が遠くてマッチも多いペア（Top 10）:")
good_pairs = []

with h5py.File(matches_file, 'r') as f:
    for key1 in f.keys():
        num1 = get_image_number(key1)
        for key2 in f[key1].keys():
            num2 = get_image_number(key2)
            matches = f[key1][key2][()]
            distance = abs(num2 - num1)
            match_count = len(matches)

            if match_count >= 500:
                good_pairs.append({
                    'pair': (key1, key2),
                    'distance': distance,
                    'matches': match_count,
                    'score': distance * (match_count / 1000)
                })

good_pairs.sort(key=lambda x: x['distance'], reverse=True)

for i, p in enumerate(good_pairs[:10], 1):
    print(f"{i:2d}. {p['pair'][0]} - {p['pair'][1]}: "
          f"distance={p['distance']:2d}, matches={p['matches']:4d}")

In [ ]:
model_path = train_gaussian_splatting(data_dir, iterations=1000)

# Step 6: Render Video
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_video = f"{OUTPUT_DIR}/gaussian_splatting_video.mp4"
success = render_video(model_path, output_video, iteration=1000)


In [ ]:
def diagnose_specific_pair(database_path, id1, id2):
    """Diagnose a specific image pair"""
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    print(f"\n🔍 Diagnosing pair {id1}-{id2}")
    print("="*60)

    # Get match info
    pair_id = id1 * 2147483648 + id2
    cursor.execute(
        "SELECT rows, F FROM two_view_geometries WHERE pair_id=?",
        (pair_id,)
    )
    result = cursor.fetchone()

    if result:
        rows, F_blob = result
        print(f"  Matches: {rows}")

        if F_blob:
            F = np.frombuffer(F_blob, dtype=np.float64).reshape(3, 3)
            print(f"  F matrix exists: {F.shape}")
            print(f"  F matrix:\n{F}")
        else:
            print("  ⚠️ F matrix is NULL!")
    else:
        print(f"  ⚠️ Pair not found in two_view_geometries!")

    # Get camera info
    cursor.execute("SELECT c.* FROM cameras c JOIN images i ON c.camera_id = i.camera_id WHERE i.image_id IN (?, ?)", (id1, id2))
    print("\n  Cameras:")
    for row in cursor.fetchall():
        cam_id, model, w, h, params_blob, prior = row
        params = np.frombuffer(params_blob, dtype=np.float64)
        print(f"    Camera {cam_id}: {w}x{h}, model={model}, params={params}")

    conn.close()
    print("="*60)

# 実行
diagnose_specific_pair('/content/output/colmap/database.db', 11, 20)

In [ ]:
import sqlite3
import numpy as np

def diagnose_database(database_path):
    """Diagnose why COLMAP can't find initial pair"""
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    print("\n🔍 Database Diagnosis")
    print("="*60)

    # Get match statistics
    cursor.execute("""
        SELECT pair_id, rows, config
        FROM two_view_geometries
        ORDER BY rows DESC
        LIMIT 10
    """)

    print("\nTop 10 matches by count:")
    for pair_id, rows, config in cursor.fetchall():
        image_id2 = pair_id % 2147483648
        image_id1 = (pair_id - image_id2) // 2147483648
        print(f"  Images {image_id1}-{image_id2}: {rows} matches, config={config}")

    # Get image and camera info
    cursor.execute("""
        SELECT i.image_id, i.name, i.camera_id, c.width, c.height
        FROM images i
        JOIN cameras c ON i.camera_id = c.camera_id
        LIMIT 5
    """)

    print("\nSample images:")
    for img_id, name, cam_id, w, h in cursor.fetchall():
        print(f"  Image {img_id}: {name}, camera {cam_id} ({w}x{h})")

    conn.close()
    print("="*60)

# 実行
diagnose_database('/content/output/colmap/database.db')